Hi All,

I trust you all having a great week so far.

Mixing it up a little for this week 😊 I have attached an exact from our Law schema.
It’s the NPS response data.

Currently LN is facing a huge problem with our Tax customers, and the attrition has risen substantially since lock-down.

I want each of us to use the attached CSV to determine if there is any value/insights we can extract from the NPS data.
The two new segments in Tax are; Mid Tax and Solo Tax.

I have attached the SQL too, in case you want to further extent the features/columns.

The ask; Load the data in pandas, and derive some insight/value – easy right 😉

Some thoughts:
How many responses do we have for the different tax segments compared to the others?
Are there any trends/insights between the different segments and product families?
Can we use natural language processing on the verbatims?
Pre and Post lock down, how did the Tax segment change?


questions for exploration:
1. count of ratings for tax (total and by prod)
2. avg ratings for tax (total and by prod)
3. driving force in tax (total and by prod)
4. count of ratings for other (total and by prod)
5. avg ratings for other (total and by prod)
6. % nps ratings catogories among segments; between tax and non tax
7. trend comparison for before and after lockdown
8. size of the segment - #cust per seg with currently active online subs (core prod)
9. compare between rate and job title info (change value to 1 and 0)
10. main reason for detractors
11. exclude unspecified cust - (no need - all rows have cust name)
12. same contact id rating through time

In [3]:
import pandas as pd
import plotly.express as plx
from sklearn.metrics import mean_squared_error
import statsmodels.api as sm
import pickle
import numpy as np

In [89]:
#df_nps = pd.read_csv('nps.csv',encoding='cp1252')

#with open('nps.pkl', 'wb') as f:
 #   pickle.dump(df_nps, f)
    
#with open('nps.pkl', 'rb') as f:
 #   raw = pickle.load(f)



In [4]:
with open('nps_data.pkl', 'rb') as f:
    raw = pickle.load(f)

In [5]:
raw['NPS']= raw['NPS'].replace('Passive', 'Passives')


In [6]:
raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30831 entries, 0 to 30830
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   DT                      30831 non-null  datetime64[ns]
 1   ACCNT_LEGCY_ID          30831 non-null  object        
 2   ACCNT_NAME              30831 non-null  object        
 3   MASTER_CUSTOMER_NAME    30513 non-null  object        
 4   MASTER_CUSTOMER_NUMBER  30513 non-null  object        
 5   CONTACT_ID              19922 non-null  float64       
 6   JOB_TITLE               30831 non-null  object        
 7   PF_NAME                 30831 non-null  object        
 8   CUST_SUB_CLASS_2        30831 non-null  object        
 9   NPS_RATING              30831 non-null  int64         
 10  NPS                     30831 non-null  object        
 11  REASON                  30831 non-null  object        
 12  SCORE_RSN               30831 non-null  object

In [7]:

base = (raw
        .rename(columns = {'CUST_SUB_CLASS_2': 'Segment'})
        .loc[~ raw['CUST_SUB_CLASS_2'].isin(['Unspecified','Internal'])] # drop internal as well
        .assign(yr_mth = lambda x: pd.to_datetime(x['DT']).dt.to_period('M'))
        .assign(lock_down = raw['DT'].apply(lambda x: "Y" if x >= pd.to_datetime('2020-03-01',format ='%Y-%m-%d') else "N"))
        .fillna(np.nan)        
        
)

In [8]:
base['Segment'].value_counts()

Other Public Sector        5075
Top Tier                   3486
General Practice           2946
Large Tax                  2429
Consumer-led               2083
Large Corporate Legal      1833
Full Service Commercial    1824
Other Tax                  1750
Small Law                  1663
Bar                         965
Mid Law                     749
Mid Tax                     668
Solo Tax                    301
Export                      228
Other Tax - Legal            70
Small Corporate Legal        53
Other Academic               39
BIS Corporate Non Legal      12
Other Tax - Corporate        12
Solo                          6
Law Schools                   2
Trade                         1
Large Tax - Corporate         1
Name: Segment, dtype: int64

In [10]:
base.describe()

,CONTACT_ID,NPS_RATING
count,1.765400e+04,26196.000000
mean,8.925999e+06,7.381585
std,2.961107e+06,2.334269
min,4.989039e+06,0.000000
25%,6.532440e+06,6.000000
50%,7.690733e+06,8.000000
75%,1.056013e+07,9.000000
max,1.617560e+07,10.000000


In [11]:
base.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26196 entries, 0 to 30830
Data columns (total 16 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   DT                      26196 non-null  datetime64[ns]
 1   ACCNT_LEGCY_ID          26196 non-null  object        
 2   ACCNT_NAME              26196 non-null  object        
 3   MASTER_CUSTOMER_NAME    26196 non-null  object        
 4   MASTER_CUSTOMER_NUMBER  26196 non-null  object        
 5   CONTACT_ID              17654 non-null  float64       
 6   JOB_TITLE               26196 non-null  object        
 7   PF_NAME                 26196 non-null  object        
 8   Segment                 26196 non-null  object        
 9   NPS_RATING              26196 non-null  int64         
 10  NPS                     26196 non-null  object        
 11  REASON                  26196 non-null  object        
 12  SCORE_RSN               26196 non-null  object

In [27]:
base['CONTACT_ID'] = base['CONTACT_ID'].astype('Int64')

In [16]:
base['PF_NAME'].value_counts()

LexisLibrary       12061
LexisPSL            7120
TolleyLibrary       5930
TolleyGuidance       739
Unspecified          213
Nexis.com (N&B)      133
Name: PF_NAME, dtype: int64

In [29]:
# table showing count of ratings for each seg by pf_name
(base
     .groupby(['PF_NAME', 'Segment'])[['NPS_RATING']].count().unstack()
     .fillna(0)
     .astype('int64')
  
)

NPS_RATING                           \
Segment         BIS Corporate Non Legal  Bar Consumer-led Export   
PF_NAME                                                            
LexisLibrary                          4  913          718    211   
LexisPSL                              8   39         1348     14   
Nexis.com (N&B)                       0    0            0      0   
TolleyGuidance                        0    0            0      0   
TolleyLibrary                         0    0            0      3   
Unspecified                           0   13           17      0   

                                                          \
Segment         Full Service Commercial General Practice   
PF_NAME                                                    
LexisLibrary                       1081             1328   
LexisPSL                            724             1579   
Nexis.com (N&B)                       1                2   
TolleyGuidance                        0                0   
TolleyLibrary                         4                5   
Unspecified                          14               32   

                                                                       \
Segment         Large Corporate Legal Large Tax Large Tax - Corporate   
PF_NAME                                                                 
LexisLibrary                      691       249                     0   
LexisPSL                         1041         5                     0   
Nexis.com (N&B)                    29         1                     0   
TolleyGuidance                      0       363                     1   
TolleyLibrary                      57      1798                     0   
Unspecified                        15        13                     0   

                             ...                                \
Segment         Law Schools  ... Other Public Sector Other Tax   
PF_NAME                      ...                                 
LexisLibrary              0  ...                2560       328   
LexisPSL                  0  ...                 578        27   
Nexis.com (N&B)           0  ...                  26         3   
TolleyGuidance            0  ...                  64       134   
TolleyLibrary             0  ...                1821      1250   
Unspecified               2  ...                  26         8   

                                                                               \
Segment         Other Tax - Corporate Other Tax - Legal Small Corporate Legal   
PF_NAME                                                                         
LexisLibrary                        1                30                    16   
LexisPSL                            6                21                    32   
Nexis.com (N&B)                     2                10                     0   
TolleyGuidance                      0                 0                     1   
TolleyLibrary                       3                 8                     4   
Unspecified                         0                 1                     0   

                                                        
Segment         Small Law Solo Solo Tax Top Tier Trade  
PF_NAME                                                 
LexisLibrary          719    3       46     2669     1  
LexisPSL              863    3        7      790     0  
Nexis.com (N&B)        51    0        0        8     0  
TolleyGuidance          0    0       54        0     0  
TolleyLibrary           6    0      194        3     0  
Unspecified            24    0        0       16     0  

[6 rows x 23 columns]

In [44]:
# table showing % count by PF for different seg
(base
     .groupby(['Segment', 'PF_NAME'])[['NPS_RATING']].count().unstack()
     #.assign(num_response = lambda x: x.groupby(['Segment', 'PF_NAME'])[['NPS_RATING']].count())
     .fillna(0)
     .astype('int64')
   #  .assign(subtotal = lambda x: x.groupby(['Segment']).NPS_RATING.transform('count'))
        
)


NPS_RATING                                          \
PF_NAME                 LexisLibrary LexisPSL Nexis.com (N&B) TolleyGuidance   
Segment                                                                        
BIS Corporate Non Legal            4        8               0              0   
Bar                              913       39               0              0   
Consumer-led                     718     1348               0              0   
Export                           211       14               0              0   
Full Service Commercial         1081      724               1              0   
General Practice                1328     1579               2              0   
Large Corporate Legal            691     1041              29              0   
Large Tax                        249        5               1            363   
Large Tax - Corporate              0        0               0              1   
Law Schools                        0        0               0              0   
Mid Law                          449       33               0              1   
Mid Tax                           32        1               0            121   
Other Academic                    12        1               0              0   
Other Public Sector             2560      578              26             64   
Other Tax                        328       27               3            134   
Other Tax - Corporate              1        6               2              0   
Other Tax - Legal                 30       21              10              0   
Small Corporate Legal             16       32               0              1   
Small Law                        719      863              51              0   
Solo                               3        3               0              0   
Solo Tax                          46        7               0             54   
Top Tier                        2669      790               8              0   
Trade                              1        0               0              0   

                                                   
PF_NAME                 TolleyLibrary Unspecified  
Segment                                            
BIS Corporate Non Legal             0           0  
Bar                                 0          13  
Consumer-led                        0          17  
Export                              3           0  
Full Service Commercial             4          14  
General Practice                    5          32  
Large Corporate Legal              57          15  
Large Tax                        1798          13  
Large Tax - Corporate               0           0  
Law Schools                         0           2  
Mid Law                           263           3  
Mid Tax                           510           4  
Other Academic                      1          25  
Other Public Sector              1821          26  
Other Tax                        1250           8  
Other Tax - Corporate               3           0  
Other Tax - Legal                   8           1  
Small Corporate Legal               4           0  
Small Law                           6          24  
Solo                                0           0  
Solo Tax                          194           0  
Top Tier                            3          16  
Trade                               0           0

In [26]:
# table showing avg ratings for each seg by pf_name
(base
     .groupby(['PF_NAME','Segment'])[['NPS_RATING']].mean().unstack()
     .fillna(0)
     .astype('int64')
)

NPS_RATING                          \
Segment         BIS Corporate Non Legal Bar Consumer-led Export   
PF_NAME                                                           
LexisLibrary                          8   6            6      8   
LexisPSL                              6   8            7      7   
Nexis.com (N&B)                       0   0            0      0   
TolleyGuidance                        0   0            0      0   
TolleyLibrary                         0   0            0      7   
Unspecified                           0   6            7      0   

                                                          \
Segment         Full Service Commercial General Practice   
PF_NAME                                                    
LexisLibrary                          7                7   
LexisPSL                              7                7   
Nexis.com (N&B)                       7               10   
TolleyGuidance                        0                0   
TolleyLibrary                         4                6   
Unspecified                           7                7   

                                                                       \
Segment         Large Corporate Legal Large Tax Large Tax - Corporate   
PF_NAME                                                                 
LexisLibrary                        6         6                     0   
LexisPSL                            7         8                     0   
Nexis.com (N&B)                     7         7                     0   
TolleyGuidance                      0         8                     6   
TolleyLibrary                       6         7                     0   
Unspecified                         7         8                     0   

                             ...                                \
Segment         Law Schools  ... Other Public Sector Other Tax   
PF_NAME                      ...                                 
LexisLibrary              0  ...                   7         7   
LexisPSL                  0  ...                   7         6   
Nexis.com (N&B)           0  ...                   6         7   
TolleyGuidance            0  ...                   8         7   
TolleyLibrary             0  ...                   8         7   
Unspecified              10  ...                   8         8   

                                                                               \
Segment         Other Tax - Corporate Other Tax - Legal Small Corporate Legal   
PF_NAME                                                                         
LexisLibrary                        5                 6                     6   
LexisPSL                            7                 6                     6   
Nexis.com (N&B)                     4                 8                     0   
TolleyGuidance                      0                 0                     9   
TolleyLibrary                       7                 5                     7   
Unspecified                         0                 5                     0   

                                                        
Segment         Small Law Solo Solo Tax Top Tier Trade  
PF_NAME                                                 
LexisLibrary            6    5        7        7     3  
LexisPSL                7    4        7        7     0  
Nexis.com (N&B)         7    0        0        8     0  
TolleyGuidance          0    0        7        0     0  
TolleyLibrary           6    0        7        5     0  
Unspecified             6    0        0        7     0  

[6 rows x 23 columns]